In [38]:
import csv
import json
from datetime import datetime
"""
    3. *job_compatibility*
        From "articoli_macchine.json" we retrieve the associations
    4. *machine_compatibility*
        From "macchine_articoli.json" we retrieve the associations
    """
job_compatibility = {} #
machine_compatibility = {} #DONE
id_articles = {} #Dictionary of associations for giving at each articles a numerical id

with open('data/utils/articoli_macchine.json') as file:
    job_compatibility = json.load(file)

with open('data/utils/macchine_articoli.json') as file:
    machine_compatibility = json.load(file)

for idx, article in enumerate(job_compatibility):
    id_articles[article] = idx
print(f'id_articles: {id_articles}')
print(f'number of machines: {len(machine_compatibility)}\nnumber of articles: {len(job_compatibility)}')


def date_hours_parser(start_date, due_date):
    """
    Parser for start and due date from strings to hours
    """
    # Parse input dates
    start = datetime.strptime(start_date, "%Y-%m-%d")
    due = datetime.strptime(due_date, "%Y-%m-%d")
    
    # Get current date and time
    current_datetime = datetime.now()

    # Calculate differences in hours
    start_hours = int((start - current_datetime).total_seconds() // 3600)
    due_hours = int((due - current_datetime).total_seconds() // 3600)

    if start_hours < 0:
        start_hours = 0

    return start_hours, due_hours


id_articles: {'2277DN mini': 0, '2011DN mini': 1, '2271DN 20700 mini': 2, '749DN mini': 3, 'X22-1703DN minispole': 4, 'X23-3399S+ZF': 5, '2270s+z rpm 19800': 6, '2270S+ZN 19800': 7, 'X23-2299V5ZF': 8, '7032S+ZN': 9, '2270s+z rpm 21000': 10, '2250S+ZN': 11, '31002DN1': 12, 'X24-4401S+ZF2': 13, 'X23-4493S+ZF': 14, 'X21-2291SF': 15, 'X21-2291sf': 16, '2270S+ZN mini': 17, 'X21-2099S+ZN mini': 18, 'X24-2001S+ZN': 19, '13007ZN/F-14500': 20, '28503ZN': 21, '2246ZN 10800': 22, 'X23-2280S+ZN': 23, 'X20-2222DN2 mini': 24, '33002DN': 25, 's22-700011DN': 26, 'S600A04DN': 27, 'S600A03DN': 28, '33001DN2': 29, 'X23-8397DN': 30, '31003DN220274': 31, 'X21-39599V7DN0332': 32, '62002DN': 33, '39502DN-14446': 34, '39502dn': 35, '62002DN2': 36, 'X21-20099DN2': 37, 'S180001DN': 38, 'X23-47097DN366': 39, 'X24-31001DN366': 40, 'X23-47097DN1': 41, '13005ZF-14000': 42, '2265SN2': 43, 'X24-2202s+z': 44, '7034ZN': 45, '2275S+ZN-B10A': 46, 'X22-2211V2zf': 47, '4407DN': 48, 'X22-7810S-ZN354 4 ci': 49, 'X22-2203S+ZN

In [39]:
class Product (object) :
    '''
    A Product is a request for a certain amount of an certain article
    '''
    def __init__(self, id, article, kg_request, start_date, due_date):
        # Input level Data
        self.id = id
        self.article = article
        self.kg_request = kg_request
        self.start_date = start_date
        self.due_date = due_date

        # Output data (will be filled by the scheduler output)
        self.machine = {}
        self.velocity = {}
        self.num_levate = {}
        self.setup_beg = {}
        self.setup_end = {}
        self.cycle_end = {}
        self.load_beg = {}
        self.load_end = {}
        self.unload_beg = {}
        self.unload_end = {}
    
    def __str__(self) :
        return f"Product {self.id}\n    Article : {self.article}\n    Request : {self.kg_request} Kg\n    Start date : {self.start_date}\n    Due date : {self.due_date}\n---"

class RunningProduct (Product) :
    '''
    A RunningProduct is a some product which is already being processed by some machine
    when the scheduling operation begins
    '''
    def __init__(self, id, article, kg_request, start_date, due_date, machine, operator, velocity, remaining_levate, current_op_type, remaining_time) :
        super().__init__(id, article, kg_request, start_date, due_date)
        # Data related to settings associated to the running product
        self.operator = operator
        self.machine[0] = machine
        self.velocity[0] = velocity
        self.remaining_levate = remaining_levate # amount of levate to finish the cycle

        # states in which phase the product is when the scheduling starts
        #   can have 4 possible values {0, 1, 2, 3} 
        #   associated to a all cycle's possible phases {setup, load, running, unload}
        self.current_op_type = current_op_type
        # remaining time to finish the active production phase
        self.remaining_time = remaining_time

    def __str__(self) :
        return f"RunningProduct {self.id}\n    Article : {self.article}\n    Request : {self.kg_request} Kg\n    Start date : {self.start_date}\n    Due date : {self.due_date}\n    Machine : {self.machine}\n    Operator : {self.operator}\n    Velocity : {self.velocity}\n    Remaining levate : {self.remaining_levate}\n    Current operation type : {self.current_op_type}\n    Remaining time : {self.remaining_time}\n---"


class Schedule(object):
    '''
    A schedule is the final result of the Job Scheduling Problem (JSP).
    '''
    def __init__(self, products):
        # Reference to products class instances, ensuring proper types
        self.products = products
        for _, prod in self.products:
            if not isinstance(prod, (Product, RunningProduct)):
                raise ValueError("Invalid product type")

    def __str__(self):
        output = "Production Schedule:\n"
        for p, prod in self.products :
            output += f"Product : {chr(p+65)}\n"
            for c in prod.setup_beg.keys():
                output += f"    Cycle {c} :\n"
                output += f"        Machine   : {prod.machine[c]}:\n"
                output += f"        Velocity  : {prod.velocity[c]}\n"
                output += f"        Cycle End : {prod.cycle_end[c]}\n"
                output += f"        Setup     : ({prod.setup_beg[c]}, {prod.setup_end[c]})\n"
                for l in range(prod.num_levate[c]) :
                    if (c,l) in prod.load_beg.keys() : 
                        output += f"            Levata [{l}] : ({prod.load_beg[c,l]}, {prod.load_end[c,l]}) => ({prod.unload_beg[c,l]}, {prod.unload_end[c,l]})\n"
            output += "\n"
        
        return output
    
    def __len__(self):
        return len(self.products)

In [ ]:

def init_csv_data(csv_path: str, costs = (2,1,1), running_products =  {}, base_setup_cost = 2):
    """
    1. *common_products*
        The csv gave us:
        - Client name (useless)
        - Product code 
        - Product quantity
        - Insertion date
        - Due date
        for every row, which is a "Product" object and the list of those form the "common_products"
    2. *running_products*
        Assuming an empty list right now
    3. *job_compatibility*
        From "articoli_macchine.json" we retrieve the associations
    4. *machine_compatibility*
        From "macchine_articoli.json" we retrieve the associations
    5. *base_setup_cost*
        Costant taken from input
    6. *base_load_cost*
        For every product is machine dependent(depends from the number of fusi) 
    7. *base_unload_cost*
        Same as point 6 
    8.  *base_levata_cost*
        For every product, from "lista_articoli" see "ore_levata" 
    9. *standard_levate*
        For every product, form "lista_articoli" see "no_cicli" but is leavta 
    10. *kg_per_levata*
        See from "lista_articoli" the "kg_ora" and the "ore_levata" 
    
"""
    base_setup_cost, base_load_cost, base_unload_cost = costs
    with open(csv_path, newline='') as csvfile:
        
        csv_data = csv.reader(csvfile, delimiter=',', quotechar='|')
        common_products = []
        str_out = ""
        for idx, row in enumerate(csv_data):
            if idx > 0:
                str_out = str_out + ', '.join(row) 
                str_out += '\n'
                # print(f'start_date:  {row[3]} - type: {type(row[3])}')
                start_date, due_date = date_hours_parser(row[3], row[4])
                common_products.append(Product(idx, id_articles[row[1]], int(float(row[2])), start_date, due_date))

        
    print(str_out)
    for i in common_products:
        print(i.__str__())
# common_products = []
# product_id = 0
# for _ in range(num_common_jobs) :
#     article = random.randint(0, len(articles)-1)
#     kg_request = random.randint(10, 40)
#     start_date = 0
#     due_date = horizon
#     common_products.append(Product(product_id, article, kg_request, start_date, due_date))
#     product_id += 1

        
    # return common_products, running_products, job_compatibility, machine_compatibility, base_setup_cost, base_load_cost, base_unload_cost, base_levata_cost, standard_levate, kg_per_levata

base_setup_cost = {}

CSV_PATH = 'data/new_orders.csv'



In [45]:

init_csv_data(CSV_PATH)


CALZIFICIO ATHENA, 4463S+ZN1, 420.0, 2024-11-12, 2024-12-12
FEIN ELAST GRABHER, 10353ZF, 380.0, 2024-11-12, 2024-12-08
FISHER & PAYKEL, X21-39599V7DN0332, 195.0, 2024-11-12, 2025-01-09
TATRASVIT, S18-200014DN, 7.0, 2024-11-12, 2024-12-08
TATRASVIT, S18-200014DN, 7.0, 2024-11-12, 2024-12-08
CALZIFICIO GENNY, X23-7892DN mini, 196.0, 2024-11-12, 2024-12-08
INVENTEX, S15-160018DN, 587.0, 2024-11-12, 2024-12-02
CALZIFICIO BRAM, X24-2001S+ZN, 115.0, 2024-11-12, 2024-11-23
FEIN ELAST GRABHER, 10353ZF, 380.0, 2024-11-12, 2024-12-08
INVENTEX, S15-160018DN, 587.0, 2024-11-12, 2024-12-02

[<__main__.Product object at 0x7ff8ec5f8070>, <__main__.Product object at 0x7ff8ec736610>, <__main__.Product object at 0x7ff8ec2f8b20>, <__main__.Product object at 0x7ff8ec5712b0>, <__main__.Product object at 0x7ff8ec571fa0>, <__main__.Product object at 0x7ff8ec230fd0>, <__main__.Product object at 0x7ff8ec2305b0>, <__main__.Product object at 0x7ff8ec230580>, <__main__.Product object at 0x7ff8ec2302e0>, <__main__

In [42]:
from datetime import datetime

def calculate_hours_difference(start_date, due_date):
    # Parse input dates
    start = datetime.strptime(start_date, "%Y-%m-%d")
    due = datetime.strptime(due_date, "%Y-%m-%d")
    
    # Get current date and time
    current_datetime = datetime.now()

    # Calculate differences in hours
    start_hours = int((start - current_datetime).total_seconds() // 3600)
    due_hours = int((due - current_datetime).total_seconds() // 3600)

    if start_hours < 0:
        start_hours = 0

    return start_hours, due_hours

# Example usage
start_date = "2024-11-12"
due_date = "2024-12-01"
hours_until_start, hours_until_due = calculate_hours_difference(start_date, due_date)

print(f"Hours until start date: {hours_until_start}")
print(f"Hours until due date: {hours_until_due}")


Hours until start date: 0
Hours until due date: 436
